In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timezone
import glob
import os
from tqdm import tqdm
import codecs
import csv

import pandas as pd

In [2]:
##Read in Excel File

file="/mnt/hgfs/shared/cloud billing/ITOPSv22024_2025_part001.csv"

# Load spreadsheet
df_billing = pd.read_csv(file)


In [27]:
#df_billing.head()
df_test = df_billing[df_billing['organization_accountname'] == 'dev_matt._vangee']

In [57]:
# Step 1: Drop 'sum_lineitem_usageamount' column
df = df_billing.drop(columns=['sum_lineitem_usageamount'])

# Step 2: Filter rows where 'bill_invoiceid' starts with 'EUINI'
df = df_billing[df['bill_invoiceid'].str.startswith('EUINI', na=False)]

# Step 3: Prepare to calculate 'island_networks_cost' and 'customer_cost'
# Pivot data to have lineitem_lineitemtype as columns and their corresponding costs as values
df_pivoted = df.pivot_table(
    index=['organization_accountname', 'lineitem_usageaccountid', 'month'],
    columns='lineitem_lineitemtype',
    values='sum_lineitem_unblendedcost',
    fill_value=0
).reset_index()

# Step 4: Calculate 'island_networks_cost' and 'customer_cost'
# Note: Using get() method with 0 default for cases where columns might be missing

# 'island_networks_cost' is 'Usage' - 'SppDiscount'
df_pivoted['island_networks_cost'] = (
    df_pivoted['Usage'] + df_pivoted.get('SppDiscount', 0)    
     + df_pivoted.get('BundledDiscount', 0)
     + df_pivoted.get('RIFee', 0)
    )

# 'customer_cost' is 'Usage' + 'SavingsPlanNegation' + 'BundledDiscount' + 'Credit' + 'RIfee'
df_pivoted['customer_cost'] = (
    df_pivoted['Usage'] +  df_pivoted['SavingsPlanCoveredUsage']
    + df_pivoted.get('RIFee', 0)
    + df_pivoted.get('SavingsPlanNegation', 0)
    + df_pivoted.get('BundledDiscount', 0)
    #- df_pivoted.get('Credit', 0)
)

# Display the resulting DataFrame
df_pivoted.round(3)


lineitem_lineitemtype,organization_accountname,lineitem_usageaccountid,month,BundledDiscount,Credit,DiscountedUsage,Fee,RIFee,SavingsPlanCoveredUsage,SavingsPlanNegation,SavingsPlanRecurringFee,SavingsPlanUpfrontFee,SppDiscount,Tax,Usage,island_networks_cost,customer_cost
0,258579678158,258579678158,2024-09,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,-0.308,2.29,10.271,9.963,10.271
1,258579678158,258579678158,2024-10,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,-0.313,2.32,10.439,10.126,10.439
2,526068689859,526068689859,2024-09,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,-0.305,2.28,10.181,9.876,10.181
3,526068689859,526068689859,2024-10,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,-0.316,2.34,10.531,10.215,10.531
4,637423230591,637423230591,2024-09,-32.606,0.0,0.0,0.0,0.0,0.504,-0.504,0.0,0.0,-25.398,188.87,879.211,821.207,846.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,awstransitvpc,510595481552,2024-10,0.000,0.0,0.0,0.0,0.0,0.205,-0.205,0.0,0.0,-36.373,270.49,1212.430,1176.057,1212.430
117,dev-Samuel.Zimmerman,958937274050,2024-09,-32.400,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,-13.773,102.42,491.513,445.340,459.113
118,dev-Samuel.Zimmerman,958937274050,2024-10,-33.480,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,-14.233,105.83,507.919,460.205,474.438
119,dev_matt._vangee,949223504077,2024-09,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,-0.304,2.26,10.121,9.817,10.121


In [59]:
# Filter the pivoted DataFrame for a specific month
pivot_df_filtered = df_pivoted[df_pivoted['month'] == '2024-09']

# Display the filtered pivoted DataFrame
pivot_df_filtered.round(3)


lineitem_lineitemtype,organization_accountname,lineitem_usageaccountid,month,BundledDiscount,Credit,DiscountedUsage,Fee,RIFee,SavingsPlanCoveredUsage,SavingsPlanNegation,SavingsPlanRecurringFee,SavingsPlanUpfrontFee,SppDiscount,Tax,Usage,island_networks_cost,customer_cost
0,258579678158,258579678158,2024-09,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,-0.308,2.29,10.271,9.963,10.271
2,526068689859,526068689859,2024-09,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,-0.305,2.28,10.181,9.876,10.181
4,637423230591,637423230591,2024-09,-32.606,0.000,0.0,0.0,0.000,0.504,-0.504,0.0,0.0,-25.398,188.87,879.211,821.207,846.605
6,654654565879,654654565879,2024-09,-32.400,0.000,0.0,0.0,0.000,1.469,-1.469,0.0,0.0,-115.570,859.45,3884.742,3736.772,3852.342
9,767397748348,767397748348,2024-09,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,-0.037,0.27,1.223,1.186,1.223
11,851725533419,851725533419,2024-09,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,-5.767,42.89,192.247,186.480,192.247
13,931617126933,931617126933,2024-09,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,-0.085,0.63,2.825,2.740,2.825
15,973648169211,973648169211,2024-09,-100.576,0.000,0.0,0.0,0.000,3.840,-3.840,0.0,0.0,-120.406,895.42,4114.126,3893.143,4013.550
17,975050056716,975050056716,2024-09,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,-35.756,265.90,1191.868,1156.112,1191.868
19,BTR Certificate Authority,58264552935,2024-09,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.0,0.0,-12.040,89.54,401.344,389.304,401.344
